In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from eventlet.timeout import Timeout
import random
import time

# import auxillary packages
import requests  # for loading the example source code
import openai
import json

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from flaml.autogen.code_utils import extract_code
from create_bench_agent import CreateBenchAgent



In [2]:
example_qa = """
----------
Param Update: ```JSON
"end_charge_time": 9```
Example User request: Charge the car till 9 AM.
----------
Param Update: ```JSON
"end_charge": 1.00```
Example Request: Charge the car to full charge.
----------
Param Update: ```JSON
"end_charge": 1.00```
Example Request: Charge the car to full charge.
"""

In [3]:
params_filepath = "/Users/viditjain/Desktop/LLM Optimizer/EnergySaverLLM/EnergySaverLLM/Model/params/EVCharging.json"

In [35]:
log_history = {}

In [36]:
autogen.oai.ChatCompletion.start_logging(log_history)

In [37]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": [ "gpt-4","gpt-3.5-turbo-16k"],
    },
)

In [38]:
agent = CreateBenchAgent(
    name="CreateBenchAgent",
    example_qa=example_qa,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

In [39]:
with open('benchmark_values.json', 'r') as f:
    # text = f.read()
    values_db = json.loads(f.read())

In [40]:
param_update_single = """```JSON
"{key}": {new_val}
```"""

In [41]:
def getRange(args):
    return [round(x * args[3], 2) for x in np.arange(args[0], args[1], args[2], dtype=int)]


In [42]:
benchmark_dataset = []
num_elems = 20
for key, val_range in values_db.items():
    val_list = random.sample(list(getRange(val_range)), min(num_elems, len(list(getRange(val_range)))))
    for new_val in val_list:
        benchmark_instance = {}
        benchmark_instance["json_str"] = param_update_single.format(key = key, new_val = new_val)
        agent = CreateBenchAgent(
            name="CreateBenchAgent",
            example_qa=example_qa,
            json_filepath=params_filepath,
            llm_config={
                "request_timeout": 600,
                "seed": 42,
                "config_list": config_list,
            }
        )
        user = UserProxyAgent(
            "user", max_consecutive_auto_reply=0,
            human_input_mode="NEVER", code_execution_config=False
        )
        
        user.initiate_chat(agent, message = benchmark_instance["json_str"], silent = True, clear_history = True)

        log_list = []
        log_list = eval(list(log_history.keys())[-1])
        benchmark_instance['prompt'] = ""+(log_list[-1]['content'])

        benchmark_dataset.append(benchmark_instance)
        assert len(benchmark_dataset) < 2 or benchmark_dataset[-1]['prompt'] != benchmark_dataset[-2]['prompt']
        
        time.sleep(0.5)
        print(len(benchmark_dataset))

CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Increase the weightage of carbon emissions cost in total cost calculation to 0.85.

--------------------------------------------------------------------------------
1
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Increase the weight of carbon cost in total cost calculation to 0.7.

--------------------------------------------------------------------------------
2
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Increase the weightage of carbon cost in total cost calculation to 0.35.

--------------------------------------------------------------------------------
3
CreateBenchAg

In [27]:
benchmark_dataset

[{'json_str': '```JSON\n"carbon_cost_weight": 0.8\n```',
  'prompt': 'Charge the car till 3 PM.'},
 {'json_str': '```JSON\n"carbon_cost_weight": 0.45\n```',
  'prompt': 'Charge the car till 3 PM.'}]

In [ ]:
benchmark_combined = benchmark_dataset.copy()

In [43]:
for entry in benchmark_dataset:
    print(entry['prompt'])

Increase the weightage of carbon emissions cost in total cost calculation to 0.85.
Increase the weight of carbon cost in total cost calculation to 0.7.
Increase the weightage of carbon cost in total cost calculation to 0.35.
Reduce the weightage of carbon cost in total cost calculation to 15%.
Lower the importance of carbon cost in the total cost calculation to 25%.
Increase the weightage of carbon cost in total cost calculation to 0.5.
Adjust the weightage of the carbon cost in the total cost to 20%.
Increase the weight of carbon cost in total cost calculation to 0.75.
Increase the weightage for carbon cost in the total cost calculation to 1.1
Lessen the weightage of carbon emissions in my total cost to 0.05.
I want to reduce the weightage of carbon cost in total cost calculation to 10%.
Increase the weight of carbon cost in total cost calculation to 0.95.
Increase the weight of carbon cost in total cost calculation to 0.6.
Increase the weightage of the carbon cost in the total cost t

In [44]:
benchmark_dataset_filename = "EV_single.benchmark.json"
with open(benchmark_dataset_filename, "w") as text_file:
    text_file.write(json.dumps(benchmark_dataset, indent=2))

In [71]:
param_update_multiple = """```JSON
"{key1}": {val1},
"{key2}": {val2}
```"""

In [78]:
params_to_change = list(values_db.keys())
num_elems = 7

In [79]:
benchmark_dataset = []
for i in range(len(params_to_change)):
    for j in range(i+1, len(params_to_change)):
        key1 = params_to_change[i]
        key2 = params_to_change[j]
        
        val_range1 = values_db[key1]
        val_range2 = values_db[key2]

        val_list1 = random.sample(list(getRange(val_range1)), min(num_elems, len(list(getRange(val_range1)))))
        val_list2 = random.sample(list(getRange(val_range2)), min(num_elems, len(list(getRange(val_range2)))))
        for new_val1 in val_list1:
            for new_val2 in val_list2:
                benchmark_instance = {}
                benchmark_instance["json_str"] = param_update_multiple.format(key1 = key1, 
                                                                            val1 = new_val1,
                                                                            key2 = key2,
                                                                            val2 = new_val2)
                
                user.initiate_chat(agent, message = benchmark_instance["json_str"], 
                                            silent = True, clear_history = True)

                log_list = eval(list(log_history.keys())[-1])
                benchmark_instance['prompt'] = ""+(log_list[-1]['content'])

                benchmark_dataset.append(benchmark_instance)
                time.sleep(0.5)
                print(len(benchmark_dataset))

CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Increase the weightage of carbon cost to 0.45 and change the maximum electricity cost allowed in a charge cycle to 59.

--------------------------------------------------------------------------------
1
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Boost the carbon cost weight to 0.45 and lower my maximum electricity cost for one charge cycle to 11 dollars.

--------------------------------------------------------------------------------
2
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Reduce the maximum energy cost while charging to 12 USD and increase the weight of carbon co

In [80]:
benchmark_dataset_filename = "EV_multiple.benchmark.json"
with open(benchmark_dataset_filename, "w") as text_file:
    text_file.write(json.dumps(benchmark_dataset, indent=2))

In [ ]:
benchmark_combined.extend(benchmark_dataset)

In [ ]:
i = 0
for benchmark_instance in benchmark_combined:
    benchmark_instance['index'] = i
    i = i + 1


In [ ]:
benchmark_dataset_filename = "EV_combined.benchmark.json"
with open(benchmark_dataset_filename, "w") as text_file:
    text_file.write(json.dumps(benchmark_combined, indent=2))